# Sailpoint IdentityIQ

> http://localhost:8080/identityiq/doc/javadoc/

> http://localhost:8080/identityiq/doc/help/help/

> http://localhost:8080/identityiq/

> http://localhost:8080/identityiq/debug/

> http://localhost:8080/identityiq/define/applications/application.jsf?appId=c0a86602920d1a5a81920d7ca2d1000c

## Sailpoint IIQ 8.3 -> Beanshell 2

> https://beanshell.github.io/manual/contents.html

- không lambda `x -> x.doSomething();`
- không local variable type interface `var x = new Object();` - https://openjdk.org/projects/amber/guides/lvti-style-guide
- không varargs `doSomething(Object ...args)`
- không method reference `Class::getClass`
- không cần khai báo kiểu - Loosely Typed Java Syntax - có thể `hashtable = new Hashtable();` thay vì `Hashtable hashtable = new Hashtable();`
- ...

## Sample

### Reference Library

```xml
<?xml version='1.0' encoding='UTF-8'?>
<!DOCTYPE Rule PUBLIC "sailpoint.dtd" "sailpoint.dtd">
<Rule created="" id="c0a801fe921011358192108dcdf50076" language="beanshell" name="Reference Library">
  <Source>
    import java.lang.reflect.Field;
    import java.lang.reflect.Method;
    import java.util.ArrayList;
    import java.util.Arrays;
    import java.util.List;
    import java.util.stream.Collectors;

    public class Reference {
      protected final Object ref;
      public Reference(Object ref) {
        this.ref = ref;
      }

      public Class getRefClass() {
        return this.ref.getClass();
      }

      public Field getField(String name) throws Exception {
        return this.ref.getClass().getField(name);
      }

      public Method getMethod(String name) throws Exception {
        return this.ref.getClass().getMethod(name);
      }

      public Method getMethod(String name, Class[] parameterTypes) {
        return this.ref.getClass().getMethod(name, parameterTypes);
      }

      public Method[] getMethods(String name) throws Exception {
        Method[] methods = this.ref.getClass().getDeclaredMethods();
        List result = new ArrayList();
        for (Method method: methods) {
          if (method.getName().equals(name))
            result.add(method);
        }
        return result.toArray(new Method[0]);
      }

      public Class getType(String name) throws Exception {
        return this.getField(name).getType();
      }

      public Object getValue(String name) throws Exception {
        return this.getField(name).get(this.ref);
      }

      public void setValue(String name, Object value) throws Exception {
        this.getField(name).set(this.ref, value);
      }

      public String[] methodDefines(String name) throws Exception {
        Method[] methods = this.getMethods(name);
        List result = new ArrayList();
        for (Method method: methods) {
          result.add(methodDefine(method));
        }
        return result.toArray(new String[0]);
      }

      public static String methodDefine(Method method) {
        String methodName = method.getName();
        Class[] parameterTypes = method.getParameterTypes();
        List typeNames = new ArrayList();
        for (Class c: parameterTypes) {
          typeNames.add(c.getSimpleName());
        }
        String returnType = method.getReturnType().getSimpleName();
        return methodName + "(" + String.join(", ", typeNames) + ") -> " + returnType;
      }

      public Object invoke(String name) throws Exception {
        return this
          .getRefClass()
          .getMethod(name)
          .invoke(this.ref)
        ;
      }

      public Object invoke(String name, Object[] args) throws Exception {
        List classes = new ArrayList();
        for(Object arg: args) {
          classes.add(arg.getClass());
        }
        return this
          .getRefClass()
          .getMethod(
            name,
            classes.toArray(new Class[0])
          )
          .invoke(this.ref, args)
        ;
      }

      public Field[] getAllFields() throws Exception {
        return this.ref.getClass().getDeclaredFields();
      }

      public Method[] getAllMethods() throws Exception {
        return this.ref.getClass().getDeclaredMethods();
      }

      public String[] getAllFieldNames() {
        Field[] fields = this.ref.getClass().getDeclaredFields();
        List result = new ArrayList();
        for (Field field: fields) {
          result.add(field.getName());
        }
        return result.toArray(new String[0]);
      }

      public String[] getAllMethodNames() {
        Method[] methods = this.ref.getClass().getDeclaredMethods();
        List result = new ArrayList();
        for (Method method: methods) {
          result.add(method.getName());
        }
        return result.toArray(new String[0]);
      }
    }
  </Source>
</Rule>
```

### Demo Web Service Before Test Connection Rule

```xml
<?xml version='1.0' encoding='UTF-8'?>
<!DOCTYPE Rule PUBLIC "sailpoint.dtd" "sailpoint.dtd">
<Rule created="" id="c0a86602920d1a5a81920d7dd2060010" language="beanshell" modified="" name="IAM-DEMO-WSBefore-TestConnection" type="WebServiceBeforeOperationRule">
  <Description>The Web Services connector will call this rule before performing ANY defined operation. This rule can be used to add/update values in the endpoint object before performing the operation and/or add persistent values to the application's data.
(See information about what to return for more information)</Description>
  <ReferencedRules>
    <Reference class="sailpoint.object.Rule" id="c0a801fe921011358192108dcdf50076" name="Reference Library"/>
  </ReferencedRules>
  <Signature returnType="Object">
    <Inputs>
      <Argument name="log" type="org.apache.commons.logging.Log">
        <Description>
          The log object associated with the SailPointContext.
        </Description>
      </Argument>
      <Argument name="context" type="sailpoint.api.SailPointContext">
        <Description>
          A sailpoint.api.SailPointContext object that can be used to query the database if necessary.
        </Description>
      </Argument>
      <Argument name="application">
        <Description>The application associated with the operation being processed.</Description>
      </Argument>
      <Argument name="requestEndPoint">
        <Description>The current request information containing the header, body, context url, method type, response attribute map, 
                and response code. This object can be modified directly and returned by the rule to update the endpoint information that is
                used by the current operation.
                </Description>
      </Argument>
      <Argument name="oldResponseMap">
        <Description>Earlier response object </Description>
      </Argument>
      <Argument name="restClient">
        <Description>REST Client Object</Description>
      </Argument>
      <Argument name="provisioningPlan">
        <Description>A ProvisioningPlan object containing the payload of the http request. A provisioning plan has an account request which defines the operation to be performed on the account.
                 An account request can contain multiple attribute requests and each attribute request represents an operation on a single account attribute.
                </Description>
      </Argument>
      <Argument name="partition">
        <Description>If applicable, a Partition object with the current aggregation's partitioning information.
                </Description>
      </Argument>
    </Inputs>
    <Returns>
      <Argument name="Map">
        <Description>An updated or unmodified 'requestEndPoint' object. If application object modifications are desired, create a map containing keys 'updatedEndPoint' and 'connectorStateMap' and use it as the return value;
                Within the new map, the 'updatedEndPoint' can be set to an updated or unmodified 'requestEndPoint' object. The 'connectorStateMap' will be saved as persistent values in the application definition.
                </Description>
      </Argument>
    </Returns>
  </Signature>
  <Source>
    obj = new Object() {
      public String firstName = "John", lastName = "Doe";
      public String fullName() {
        return firstName + " " + lastName;
      }
      public String doSomething() {
        return "something";
      }
      public String doSomething(String something) {
        return something;
      }
      public String doSomething(String language, String something) {
        return something;
      }
    };

    ref = new Reference(obj);
    System.out.println("--->");
    System.out.println(obj);

    System.out.println(ref.getClass());
    System.out.println(ref.getValue("firstName"));
    System.out.println(ref.getValue("lastName"));

    ref.setValue("lastName", "Cena");
    System.out.println(ref.invoke("fullName"));

    for(methodDefine: ref.methodDefines("fullName"))
      System.out.println(methodDefine);

    for(methodDefine: ref.methodDefines("doSomething"))
      System.out.println(methodDefine);

    methods = ref.getMethods("doSomething");
    for(method: methods)
      System.out.println(method);

    System.out.println(
      ref.invoke(
        "doSomething",
        new String[] {
          "Hello world"
        }
      )
    );
    System.out.println(
      ref.invoke(
        "doSomething",
        new String[] {
          "eng",
          "Hello world"
        }
      )
    );

    return requestEndPoint;
  </Source>
</Rule>
```